# Dev_Tools_Project Exploratory Data Analysys of 'vehicles_us.csv'

## Application Overview

This application seeks to analyze data of car sales in the US.

More specifically, it will illustrate the following:
A) The average miles on the odometer per vehicle based on the manufacturer
B) The correlation between Odometer reading and Days on Market
C) The condition distribution between Manufacturers with the option to filter for both 4wd and non-4wd vehicles


Libraries used:
Streamlit
Pandas
Plotly.Express


### Our Vehicle Dataset

The dataset we'll be exploring contains a wealth of information about vehicles, including their prices, model years, conditions, fuel types, odometer readings, and more. Each row represents a unique vehicle listing.

In [133]:
import streamlit as st
import pandas as pd 
import plotly.express as px

column_names= ["price", "model_year", "model", "condition", "cylinders", "fuel", "odometer", "transmission", "type", "paint_color", "is_4wd", "date_posted", "days_listed"]


df = pd.read_csv('/Users/jeffreyheller/Desktop/Dev_Tools_Project/vehicles_us.csv', sep =',', header=None, names=column_names)

df = df.iloc[2:]

df['manufacturer'] = df['model'].apply(lambda x: x.split()[0])


# the csv file was assuming the header of the file to be the first row of data. 
# the file was opened and column names were assigned above to be able to more accurately work with the data. 


## Checking for Basic Data Structure and Information


In [134]:
display(df.describe)
display(df.head(5))
display(df.shape)


<bound method NDFrame.describe of        price model_year           model  condition cylinders fuel  odometer  \
2       9400     2011.0          bmw x5       good         6  gas  145000.0   
3      25500        NaN      ford f-150       good         6  gas   88705.0   
4       5500     2013.0  hyundai sonata   like new         4  gas  110000.0   
5       1500     2003.0      ford f-150       fair         8  gas       NaN   
6      14900     2017.0    chrysler 200  excellent         4  gas   80903.0   
...      ...        ...             ...        ...       ...  ...       ...   
51522   9249     2013.0   nissan maxima   like new         6  gas   88136.0   
51523   2700     2002.0     honda civic    salvage         4  gas  181500.0   
51524   3950     2009.0  hyundai sonata  excellent         4  gas  128000.0   
51525   7455     2013.0  toyota corolla       good         4  gas  139573.0   
51526   6300     2014.0   nissan altima       good         4  gas       NaN   

      transmissio

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,manufacturer
2,9400,2011.0,bmw x5,good,6,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19,bmw
3,25500,NaN,ford f-150,good,6,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50,ford
4,5500,2013.0,hyundai sonata,like new,4,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79,hyundai
5,1500,2003.0,ford f-150,fair,8,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9,ford
6,14900,2017.0,chrysler 200,excellent,4,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28,chrysler


(51525, 14)

## Checking for Duplicates


In [135]:
# Check for duplicate rows
duplicate_rows = df[df.duplicated()]

# Display the duplicate rows
if not duplicate_rows.empty:
    display("Duplicate Rows:")
    display(duplicate_rows)
else:
    display("No duplicate rows found.")


'No duplicate rows found.'

## The following code is a redundancy for duplicate checking. 

In [136]:
# Checking for duplicates within each category of the df

display(df['manufacturer'].value_counts())
display()
display(df['paint_color'].value_counts())
display()
display(df['condition'].value_counts())
display()
display(df['fuel'].value_counts())
display()
display(df['transmission'].value_counts())
display()
display(df['type'].value_counts())
display()
display(df['paint_color'].value_counts())

manufacturer
ford             12672
chevrolet        10611
toyota            5445
honda             3485
ram               3316
jeep              3281
nissan            3208
gmc               2378
subaru            1272
dodge             1255
hyundai           1173
volkswagen         869
chrysler           838
kia                585
cadillac           322
buick              271
bmw                267
acura              236
mercedes-benz       41
Name: count, dtype: int64

paint_color
white     10029
black      7692
silver     6244
grey       5037
blue       4475
red        4421
green      1396
brown      1223
custom     1153
yellow      255
orange      231
purple      102
Name: count, dtype: int64

condition
excellent    24773
good         20145
like new      4742
fair          1607
new            143
salvage        115
Name: count, dtype: int64

fuel
gas         47288
diesel       3714
hybrid        409
other         108
electric        6
Name: count, dtype: int64

transmission
automatic    46902
manual        2829
other         1794
Name: count, dtype: int64

type
SUV            12405
truck          12353
sedan          12154
pickup          6988
coupe           2303
wagon           1541
mini-van        1161
hatchback       1047
van              633
convertible      446
other            256
offroad          214
bus               24
Name: count, dtype: int64

paint_color
white     10029
black      7692
silver     6244
grey       5037
blue       4475
red        4421
green      1396
brown      1223
custom     1153
yellow      255
orange      231
purple      102
Name: count, dtype: int64

## Looking for Gaps in DataFrame

In [137]:
df.isnull().sum()

#'is_4wd' is a binary value (yes/no) but is represented by either 1.0(yes) or 'Nan'(no)


price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
manufacturer        0
dtype: int64

In [138]:
# the data set carries negligable amounts of mercedes-benz's, many of which have missing data in the other categories. 
# for my purposes here im going to exclude all rows that include 'mercedes-benz' in the manufacturer column 
df = df[df['manufacturer'] != 'mercedes-benz']


## Adjusting Gaps in DF

### I've adjusted the "Nan" values in the "cylinder" column to have the median value based on "model" and "model_year" values. 

In [139]:
# converting the 'cylinders' column to numeric, replacing non-numeric values with NaN
df['cylinders'] = pd.to_numeric(df['cylinders'], errors='coerce')

# median cylinders for each group of model and model_year
median_cylinders = df.groupby(['model', 'model_year'])['cylinders'].transform('median')

df['cylinders'].fillna(median_cylinders, inplace=True)


### I will use the same method to fill in the missing values for "odometer" based on the median values between "condition" and "model_year" 

In [140]:
# Convert the 'odometer' column to numeric, replacing non-numeric values with NaN
df['odometer'] = pd.to_numeric(df['odometer'], errors='coerce')

# Calculate the median odometer for each group of model_year and condition
median_odometer = df.groupby(['model_year', 'condition'])['odometer'].transform('median')

# Fill NaN values in the 'odometer' column with the calculated medians
df['odometer'].fillna(median_odometer, inplace=True)


### Finally, I'll use the same method to fill the missing values for the "model_year" column using  the median grouped values of "condition" and "model" 

In [141]:
df['model_year'] = pd.to_numeric(df['model_year'], errors= 'coerce')

median_model_year = df.groupby(['condition', 'model'])['model_year'].transform('median')

df['model_year'].fillna(median_model_year, inplace=True)

In [142]:
df.isnull().sum()


price               0
model_year          6
model               0
condition           0
cylinders         385
fuel                0
odometer          549
transmission        0
type                0
paint_color      9260
is_4wd          25912
date_posted         0
days_listed         0
manufacturer        0
dtype: int64

### There were still some missing values for the 'odometer' column so I've filled the remaining missing values with median value of the entire column. 


In [143]:
# Calculate the median of the entire 'odometer' column
overall_median_odometer = df['odometer'].median()

# Fill remaining NaN values in the 'odometer' column with the overall median
df['odometer'].fillna(overall_median_odometer, inplace=True)


## Final Cleaning

### Checking for any remaining 'nan' values

In [144]:
df.isnull().sum()

price               0
model_year          6
model               0
condition           0
cylinders         385
fuel                0
odometer            0
transmission        0
type                0
paint_color      9260
is_4wd          25912
date_posted         0
days_listed         0
manufacturer        0
dtype: int64

### Dropping the remaining rows with 'nan' values in 'model_year' and 'cylinders'


In [149]:
df.dropna(subset=['model_year'], inplace=True)
df.dropna(subset=['cylinders'], inplace=True)

df.isnull().sum()

price               0
model_year          0
model               0
condition           0
cylinders           0
fuel                0
odometer            0
transmission        0
type                0
paint_color      9206
is_4wd          25719
date_posted         0
days_listed         0
manufacturer        0
dtype: int64

### Lookign for Data Types to make sure that the Data will be easy to work with.

In [145]:
df.dtypes

price            object
model_year      float64
model            object
condition        object
cylinders       float64
fuel             object
odometer        float64
transmission     object
type             object
paint_color      object
is_4wd           object
date_posted      object
days_listed      object
manufacturer     object
dtype: object

### 'Object' and 'Float' types will be converted to 'int' types where appropriate for ease with code later on.

In [150]:
df['price']=df['price'].astype(int)

df['model_year']=df['model_year'].astype(int)

df['cylinders']=df['cylinders'].astype(int)

df['odometer']=df['odometer'].astype(int)



In [ ]:
# Save the edited DataFrame to a CSV file
df.to_csv('post_eda_us_vehicle.csv', index=False)

### There are still some missing values in both the 'cylinders' and 'odometer' columns but not enought to be concerned with for analysis purposes

# Conclusions

### The DF offers full and comprehensive data regarding the inventory of vehicles in the United States. 

### Based on graphs present in the application, there are a few conclusions we can make about this data

### 1. Acura, Toyota, GMC, and Honda all have the highest average (in order) milage-per-vehicle. We might be able to make an inferrence that cars from these manufacturers have a tendency to last longer and be more reliable. Further analysis would be required to confirm this. 

### 2. There is a positive correlation between the miles on the odometer of a vehicle and the days that is spends on-market. This shouldnt come as a surprise at all. 

### 3. Further analysis of the "condition" distribution between manufacturers reveals a lot more details concerning the inference made on the first conclusion. One revelation in particular is that when the condition distribution between Honda and Toyota is illustrated, even though Honda has a higher average mileage per vehicle, Toyota has roughly 40%-50% more units in better condition either on the road or in the market. 